In [ ]:
import zea

zea.init_device()
zea.visualize.set_mpl_style()

from ulsa.ops import AntiAliasing
from active_sampling_temporal import preload_data
import matplotlib.pyplot as plt
import keras

In [ ]:
file = zea.File("/mnt/USBMD_datasets/2024_USBMD_cardiac_S51/HDF5/20240701_P1_A4CH_0001.hdf5")
dynamic_range=(-70, -28)
validation_sample_frames, scan, probe = preload_data(file, 30, "data/raw_data", cardiac=True, dynamic_range=dynamic_range)

In [ ]:
pipeline = zea.Pipeline.from_default(
    with_batch_dim=False,
    num_patches=40,
    pfield=False,
    jit_options=None
)
pipeline.append(zea.ops.Lambda(keras.ops.expand_dims, {"axis": -1}))
resize = zea.ops.Lambda(
    keras.ops.image.resize,
    {
        "size": (112, 90),
        "interpolation": "bilinear",
        "antialias": True,  # TODO: different way of antialiasing?
    },
)
pipeline.append(resize)
# pipeline.append(zea.ops.ScanConvert(order=0))

scan.resolution = 0.0001
parameters = pipeline.prepare_parameters(probe, scan)
output = pipeline(data=validation_sample_frames[0], **parameters)
image_orig = output["data"]
plt.imshow(image_orig, cmap="gray", vmin=dynamic_range[0], vmax=dynamic_range[1])

In [ ]:
pipeline = zea.Pipeline.from_default(
    with_batch_dim=False,
    num_patches=40,
    pfield=False,
    jit_options=None
)
pipeline.insert(1, zea.ops.Downsample(4))
pipeline.append(zea.ops.Lambda(keras.ops.expand_dims, {"axis": -1}))
resize = zea.ops.Lambda(
    keras.ops.image.resize,
    {
        "size": (112, 90),
        "interpolation": "bilinear",
        "antialias": True,  # TODO: different way of antialiasing?
    },
)
pipeline.append(resize)

scan.resolution = 0.0001
parameters = pipeline.prepare_parameters(probe, scan)
output = pipeline(data=validation_sample_frames[0], **parameters)
image1 = output["data"]
plt.imshow(image1, cmap="gray", vmin=dynamic_range[0], vmax=dynamic_range[1])

In [ ]:
pipeline = zea.Pipeline.from_default(
    with_batch_dim=False,
    num_patches=40,
    pfield=False,
    jit_options=None
)
pipeline.insert(1, AntiAliasing(num_taps=32))
pipeline.insert(2, zea.ops.Downsample(4))
pipeline.append(zea.ops.Lambda(keras.ops.expand_dims, {"axis": -1}))
resize = zea.ops.Lambda(
    keras.ops.image.resize,
    {
        "size": (112, 90),
        "interpolation": "bilinear",
        "antialias": True,  # TODO: different way of antialiasing?
    },
)
pipeline.append(resize)
# pipeline.append(zea.ops.ScanConvert(order=0))

scan.resolution = 0.0001
scan.n_z = 128
parameters = pipeline.prepare_parameters(probe, scan)
output = pipeline(data=validation_sample_frames[0], **parameters, factor=keras.ops.array(6))
image2 = output["data"]
plt.imshow(image2, cmap="gray", vmin=dynamic_range[0], vmax=dynamic_range[1])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 5))
axs[0].imshow(image_orig, cmap="gray", vmin=dynamic_range[0], vmax=dynamic_range[1])
axs[0].set_title("Original")
axs[1].imshow(image1, cmap="gray", vmin=dynamic_range[0], vmax=dynamic_range[1])
axs[1].set_title("With Downsample")
axs[2].imshow(image2, cmap="gray", vmin=dynamic_range[0], vmax=dynamic_range[1])
axs[2].set_title("With AntiAliasing")